## General ZAF MODEL:

In [ ]:
import math
import pandas as pd
import numpy as np
# import MAC
mac_df = pd.read_csv("Henke 1993.txt")
# !inputs: concentrations, take off angle and accelerating voltage
elements = ['Ni', 'Fe']
C = {'Ni': 0.9, 'Fe': 0.1}
Ec = {'Ni': 8.332 , 'Fe': 7.11}
A = {'Ni': 58.71 , 'Fe': 55.847}
Z = {'Ni': 28 , 'Fe': 26}

take_off_angle = 52.5
E0 = 15 # keV

# for the emission of 
emission = {'element':'Fe', 'line':'Ka'}

# calculating X for each element
print("[+] The program is considering the emission of ", emission['element'], emission['line'])
# Extracting the needed MAC from the Henke database
sample_mac = pd.DataFrame()
for idx, value in enumerate(elements):
    new_row = pd.DataFrame({'abs': [value], 'mac': [mac_df[(mac_df['zAbs'] == value) & (mac_df['zMes'] == emission['element'])][emission['line']].values[0]] })
    sample_mac = sample_mac.append(new_row, ignore_index=True)
print(sample_mac)
print(mac_df)

[+] The program is considering the emission of  Fe Ka
  abs  mac
0  Ni   94
1  Fe   68
     zAbs zMes     Ka  Kb   La   Lb   Ll   Ma   Mb
0      H    H       0   0    0    0    0    0    0
1      H    He      0   0    0    0    0    0    0
2      H    Li  10734   0    0    0    0    0    0
3      H    Be    970   0    0    0    0    0    0
4      H    B     161   0    0    0    0    0    0
...   ...  ...    ...  ..  ...  ...  ...  ...  ...
8459   U    Ra      0   0  118   67  175  864  775
8460   U    Ac      0   0  110   61  164  802  719
8461   U    Th      0   0  100   55  151  734  657
8462   U    Pa      0   0   94   51  143  689  615
8463   U    U       0   0   86  109  131  629  550

[8464 rows x 9 columns]


In [ ]:
############### Ai absorption general ############### 
# Calculating the X & sigma & h
# X
X = pd.DataFrame()
X_tot = 0
# sigma
sigma = pd.DataFrame()
sigma_tot = 0
# h
h = pd.DataFrame()
h_tot = 0
for idx, value in enumerate(elements):
    # calculating X
    new_row = pd.DataFrame({'element': [value], 'X': sample_mac[sample_mac['abs'] == value]['mac'].values[0] * (1/math.sin(math.radians(take_off_angle)))})
    X = X.append(new_row, ignore_index=True)
    # calculating X total
    X_tot = X_tot + C[value] * sample_mac[sample_mac['abs'] == value]['mac'].values[0] * (1/math.sin(math.radians(take_off_angle)))
    # Calculating sigma
    new_row = pd.DataFrame({'element': [value], 'sigma': (4.5*pow(10,5))/(pow(E0, 1.65)-pow(Ec[value], 1.65))})
    sigma = sigma.append(new_row, ignore_index=True)
    # calculating sigma total
    sigma_tot = sigma_tot + C[value] * (4.5*pow(10,5))/(pow(E0, 1.65)-pow(Ec[value], 1.65))
    # calculating h; h = 1.2 * A/Z^2
    new_row = pd.DataFrame({'element': [value], 'h': 1.2 * A[value]/pow(Z[value],2)}) 
    h = h.append(new_row, ignore_index=True)
    # calculating h total
    h_tot = h_tot + C[value] * 1.2 * A[value]/pow(Z[value],2)
# calculating the 1/f(X)
reverse_f_X = (1 + X_tot/sigma_tot)*(1 + (h_tot/(1+h_tot))*X_tot/sigma_tot)
print("Absorption correction factor for",emission['element'] , emission['line'], reverse_f_X)

Absorption correction factor for Al Ll 1.4302029335703221


In [ ]:
#### Fluorescence F ####
# correction negligible if (E-Ec) > 5keV
# The mean depth of production of fluorescence radiation is greater than that of primary radiation (electrons)
# The correction factor Ifij/Ii relates the intensity of radiation of element i produced by fluorescence by element j, Ifij, 
# to the electron-generated intensity of radiation from element i, Ii;.
# REED (1965)
# absorption jump ratio
"""
    @input: list of elements causing the parasitic fluorescence "Fluorescer"
"""
# Test data
# to remove!!!!!
# C = {'Ni': 0.9, 'Fe': 0.1}
# emission = {'element':'Fe', 'line':'Ka'}
# End test data
# inputs 
fluorescers = ['Ni']
fluo_line = ['Ka']
fluo_yield = {'Ni':0.37}
# Pij factor for the type of fluorescence occuring, if KK ( a K line fluoresces a K line) or LL fluorescence occurs Pij=1; KL or LK Pij = 4.76 for LK and 0.24 for KL
Pij = 1
# (r_i - 1)/r_i = 0.88 for K line and equal 0.75 for L line 
abs_jump_ratio = {'K':0.88, 'L': 0.75}
# Y_0 = 0.5 * ((r_i - 1)/r_i) * w_j * (A_i/A_j)
## Fe
"""
    abs_jump_ration @input
"""
# is the mass absorption coefficient of the specimen for radiation from element i,
# i emitter, j absorber
mac_sample_i = 0
for idx2,value2 in enumerate(elements):
    mac_sample_i = mac_sample_i + C[value2] * mac_df[(mac_df['zAbs'] == value2) & (mac_df['zMes'] == emission['element'])][emission['line']].values[0]

Ifij_by_Ii = 0
for idx, value in enumerate(fluorescers):
    # abs_jump_ratio used for K line since we are interested by the K line of the element
    Y_0 = 0.5 * (abs_jump_ratio['K'] * fluo_yield[value] * (A[emission['element']]/A[value]))
    Ui = E0/Ec[emission['element']]
    Uj = E0/Ec[value]
    Y_1 = pow(((Uj - 1)/(Ui - 1)), 1.67)
    
    # mac_ij is the mass absorption coefficient of element i for radiation from element j
    # mac_j_spec is the mass absorption coefficient of the specimen for radiation from element j
    # is the mass absorption coefficient of the specimen for radiation from element j.
    mac_sample_j = 0
    for idx2,value2 in enumerate(elements):
        mac_sample_j = mac_sample_j + C[value2] * mac_df[(mac_df['zAbs'] == value2) & (mac_df['zMes'] == value)][fluo_line[idx]].values[0]
    Y_2 = mac_df[(mac_df['zAbs'] == emission['element']) & (mac_df['zMes'] == value)][fluo_line[idx]].values[0]
    Y_2 = Y_2/mac_sample_j

    # Y3 account for absorption
    u = ((mac_sample_i)/(mac_sample_j)) * (1/math.sin(math.radians(take_off_angle)))
    
    # Ec is evaluated for element i
    v = (3.3 * pow(10,5))/((pow(E0,1.65) - pow(Ec[emission['element']],1.65))*(mac_sample_j))

    Y_3 = (math.log(1 + u)/u) + (math.log(1+v)/v)
    # in case the standard is a pure element
    Ifij_by_Ii = Ifij_by_Ii + C[value]*Y_0*Y_1*Y_2*Y_3*Pij
Fi = 1/(1 + Ifij_by_Ii)
print("E0, take off angle,", E0,",",take_off_angle)
print("If_Fe_Ni/I_Fe, ", Ifij_by_Ii)
print("Emitting element", emission['element'], emission['line'], 'studied,', 'Fluorescers: ', fluo_line,', Fi Correction:', Fi)

E0, take off angle, 2 , 45
If_Al_O/I_Al,  3.393451612191698e-05
Emitting element Al Ll studied, Fluorescers:  ['Ka'] , Fi Correction: 0.9999660666353903


## Z correction:

In [ ]:
#### Test data
elements = ["Al","Cu"]
C = {'Al': 0.02, 'Cu': 0.98}
Ec = {'Al': 1.56 , 'Cu': 8.98}
A = {'Al': 26.98 , 'Cu': 63.55}
Z = {'Al':13  , 'Cu': 29}
#### Atomic Number Factor Z ####
# The so-called atomic number effect in electron microprobe analysis arises from two phenomena, namely, electron backscattering and electron retardation.
# both of which depend upon the average atomic number of the target.
# ! this correction is needed if there is a difference between the average atomic number of the specimen and that of the standard
# “light elements in a heavy matrix usually yield values which are too high.” ([Goldstein et al., 1981, p. 317])

# calculating the average Z of the sample
avg_z_spec = 0
for idx, value in enumerate(elements):
    avg_z_spec = avg_z_spec + C[value] * Z[value]
print('Average atomic number of the specimen: ',avg_z_spec)
############################## for Cu Ka ############################## 
## R backscattering correction factors range from 0.5-1 (close to 1 at low atomic numbers) 
# varies with Z and overvoltage U = Eo/Ec (as decreases towards 1 few electrons are backscattered
R = np.zeros((len(elements),len(elements)))
Ri = np.zeros((len(elements),))
for idx1, value1 in enumerate(elements):
    for idx2, value2 in enumerate(elements):
        U = E0/Ec[value1]
        R1 = 8.73 * 0.001 * pow(U,3) - 0.1669 * pow(U,2) + 0.9662 * U + 0.4523
        R2 = 2.703 * 0.001 * pow(U,3) - 5.182 * 0.01 * pow(U,2) + 0.302*U -0.1836
        R3 = (0.887 * pow(U,3) - 3.44 * U * U + 9.33 * U - 6.43)/pow(U,3)
        R[idx1][idx2]= R1 - R2 * math.log(R3*Z[value2] + 25)
        Ri[idx1] = Ri[idx1] + C[value2] * R[idx1][idx2]
# index is the emitting element
Rij_df = pd.DataFrame(R, columns=elements, index=elements)
Ri = pd.DataFrame(Ri, columns=['R'], index=elements)
print("\n## Rij for the specimen:")
print(Rij_df)
print("\n## Ri for the specimen:")
print(Ri)

# Ri = sum(j) CjRij
# Ri = C[0] * Rij
# # mean ionization potential J = 9.76 * Z + 58.8 * pow(Z, -0.19) [eV] # Berger and Seltzer (1964) !!!! problems with Z < 11, by lot of authors
J = pd.DataFrame()
for idx,value in enumerate(elements):
    new_row = pd.DataFrame({'element': [value], 'J': 9.76 * Z[value] + 58.8 * pow(Z[value], -0.19)})
    J = J.append(new_row, ignore_index=True)
print("\n## J for the specimen:")
print("J, ", J)
# # S is the electron stopping power 
# # “i represents the element i which is measured and j represents each of the elements present in the standard or specimen including element i.” ([Goldstein et al., 1981, p. 319]
S = np.zeros((len(elements),len(elements)))
Si = np.zeros((len(elements),))
for idx1, value1 in enumerate(elements):
    for idx2, value2 in enumerate(elements):
        S[idx1][idx2]= (Z[value1] / (A[value1]*(E0 + Ec[value2]))) * math.log(583*(E0+Ec[value2])/J[J['element'] == value2]['J'].values[0])
        Si[idx1] = Si[idx1] + C[value2] * S[idx1][idx2]
# index is the emitting element
S = pd.DataFrame(S, columns=elements, index=elements)
Si = pd.DataFrame(Si, columns=['S'], index=elements)
print("\n## S for the specimen:")
print("S", S)
print("\n## Si for the specimen:")
print("Si", Si)

## For the standard
# Using of element Cu
print("\n## For the standard Cu:")
Si = (Z["Cu"] / (A["Cu"]*(E0 + Ec["Cu"]))) * math.log(583*(E0+Ec["Cu"])/J[J["element"] =="Cu"]["J"].values[0])
print("Si standard, ", Si)
# R Standard
U = E0/Ec["Cu"]
R1 = 8.73 * 0.001 * pow(U,3) - 0.1669 * pow(U,2) + 0.9662 * U + 0.4523
R2 = 2.703 * 0.001 * pow(U,3) - 5.182 * 0.01 * pow(U,2) + 0.302*U -0.1836
R3 = (0.887 * pow(U,3) - 3.44 * U * U + 9.33 * U - 6.43)/pow(U,3)
Ri = R1 - R2 * math.log(R3*Z["Cu"] + 25)
print("Ri standard, ", Ri)


###"""
# As discussed earlier unless the atomic number effect is corrected for, 
# analyses of heavy elements in a light element matrix (Cu in AI) generally 
# yield values which are too low (Zj > I) while analyses of light elements 
# in a heavy matrix (AI in Cu) usually yield values which are too high (Zj < I).
#"""

Average atomic number of the specimen:  10.645199999999999

## Rij for the specimen:
          Al         O
Al -7.422518 -4.360637
O   0.936519  0.966948

## Ri for the specimen:
           R
Al -5.980548
O   0.950748

## J for the specimen:
J,    element           J
0      Al  162.998366
1       O  117.688667

## S for the specimen:
S           Al         O
Al  0.465749  0.479989
O   0.483370  0.498149

## Si for the specimen:
Si            S
Al  0.472405
O   0.490278

## For the standard Al:
Si standard,  0.46574910559472465
Ri standard,  -7.422518102362616


AttributeError: 'float' object has no attribute 'iloc'

## Example of Al in pure aluminum and in alumina quantification

In [ ]:
## calculate the intensity by integrating the area under the peaks


3.8.1


NameError: name 'Z_Li' is not defined